Assignment 2
Segmenting Toronto neighbourhoods

In [93]:
# import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
# import k-means from clustering stage
from sklearn.cluster import KMeans

ModuleNotFoundError: No module named 'folium'

In [74]:
# getting data from internet
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'xml')
table = soup.find('table',{'class':'wikitable sortable'})

In [75]:
# put content into rows
rows = table.find_all('tr')
content= []
for row in rows:
    td=[]
    for t in row.find_all('td'):
        td.append(t.text.strip())
    content.append(td)
# make the dataframe    
df = pd.DataFrame(content, columns=['PostalCode', 'Borough', 'Neighborhood'])

# clean the dataframe
df = df[~df['Borough'].isnull()] 
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
df['Neighborhood'].replace('Not assigned',df['Borough'],inplace=True)
df.head()



,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [77]:
# get shape of dataframe
df.shape

(103, 3)

In [91]:
#add Geo-spatial data
df_geo= pd.read_csv("http://cocl.us/Geospatial_data")
df_geo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_geo.set_index("PostalCode")
df.set_index("PostalCode")
toronto_data=pd.merge(df, df_geo)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

# create map of Toront using latitude and longitude values
latitude= 43.6532
longitude= -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Solving environment: \ 

In [95]:
# define foursquare credentials
CLIENT_ID = 'FJXIGCVTBZOUYSU4OZGKMRY0PTHOIKZ5VUKVLGWW2NDS4MW1'
CLIENT_SECRET = 'URDFF4OEELMJS5WRB1BREORWIVJOXZNGYOOFZDPAWY3NSEEV'
VERSION = '20200513' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FJXIGCVTBZOUYSU4OZGKMRY0PTHOIKZ5VUKVLGWW2NDS4MW1
CLIENT_SECRET:URDFF4OEELMJS5WRB1BREORWIVJOXZNGYOOFZDPAWY3NSEEV


In [ ]:
toronto_data.loc[0, 'Neighborhood']

In [ ]:
:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))